In [167]:
!pip install streamlit
!pip install pyngrok

In [177]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
from pyngrok import ngrok

Overwriting app.py


In [178]:
def load_yolo_model():
    # Loading the pre-trained model (weights and configuration)
    network = cv2.dnn.readNet("/content/yolov3.weights", "/content/yolov3.cfg")
    classes_names = []
    # Loding the file that has the classes names and storing them in the array "classes"
    with open("/content/coco.names", "r") as f:
        classes_names = [line.strip() for line in f.readlines()]
    # Getting the names of each output layer
    layer_names = network.getLayerNames()
    output_layers = [layer_names[i - 1] for i in network.getUnconnectedOutLayers()]
    # Creating an rgb color for each class. It's the color of the drawn box of each detected object
    boxes_colors = np.random.uniform(0, 255, size=(len(classes_names), 3))
    return network, classes_names, output_layers, boxes_colors

In [179]:
def process_image(uploaded_file):
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    # Loading the uploaded image
    img = cv2.imdecode(file_bytes, 1)
    # Resizing the image
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    return img

In [180]:
def detect_objects(network, output_layers, img):
    img_height, img_width, _ = img.shape
    # Converting the image into a blob
    blob_obj = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0, 0, 0), True, crop=False)
    # Setting the input of the network
    network.setInput(blob_obj)
    # Processing the blob and storing the output
    outputs = network.forward(output_layers)

    # Extracting the detected objects and their information
    class_ids, confidences, boxes_info = [], [], []
    threshold = 0.5
    # Looping throw each layer's output (array)
    for output in outputs:
        # Checking the detection result of each object from this layer
        for detection in output:
            confidence_scores = detection[5:]
            class_id = np.argmax(confidence_scores)
            confidence = confidence_scores[class_id]
            if confidence > threshold:
                # Getting the coordinates of the center of the box
                center_x = int(detection[0] * img_width)
                center_y = int(detection[1] * img_height)
                # Getting the width and height of the box
                w = int(detection[2] * img_width)
                h = int(detection[3] * img_height)
                # Getting the coordinates of the top-left point of the box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                # Adding them to the boxes_info array in order to graph them later
                boxes_info.append([x, y, w, h])
                # Storing the confidence of the object and the index of it among the classes
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Making the number of boxes for each object only 1
    indices = cv2.dnn.NMSBoxes(boxes_info, confidences, 0.5, 0.4)
    return class_ids, confidences, boxes_info, indices

In [190]:
# Drawing each highlighted box
def draw_labels(img, class_ids, boxes_info, indices, classes_names, boxes_colors):
    for i in range(len(boxes_info)):
        if i in indices:
            x, y, w, h = boxes_info[i]
            label = str(classes_names[class_ids[i]])
            color = boxes_colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
    return img

In [186]:
st.title("YOLO Object Detection")
st.write("Upload an image to detect objects using YOLOv3.")
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])
if st.button("Analyze Image") and uploaded_file:
    network, classes_names, output_layers, boxes_colors = load_yolo_model()
    img = process_image(uploaded_file)
    class_ids, confidences, boxes_info, indices = detect_objects(network, output_layers, img)
    img = draw_labels(img, class_ids, boxes_info, indices, classes_names, boxes_colors)
    st.image(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), caption="Analyzed Image", use_column_width=True)
    detected_objects = set([classes_names[class_ids[i]] for i in indices])
    st.write("Detected objects from the image:", detected_objects)

In [187]:
ngrok.kill()

In [188]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [189]:
from pyngrok import ngrok
ngrok.set_auth_token("2iHymarkdwi1qongwzKPmvyGWNb_2dp8wHc6i4rTbR2vXXrXu")
public_url = ngrok.connect(8501)
print(f"Streamlit app running at: {public_url}")

Streamlit app running at: NgrokTunnel: "https://2ee6-35-199-30-20.ngrok-free.app" -> "http://localhost:8501"
